## Test AI Angent Work flow for Bank Statements

In [47]:
import os, json, time, gc

from dotenv import load_dotenv
from IPython.display import HTML, Markdown, Image, Video
from tqdm import tqdm
from openai import OpenAI, AsyncOpenAI
import asyncio
import aiohttp
import pandas as pd
import torch
from ctypes import *

#fix bug with aysncio and jupyter
import nest_asyncio # for langchain async 
nest_asyncio.apply()

import numpy as np

In [48]:
class CFG:
    OFFLINE = False #True # for Test offline environment
    USE_LLAMA3 = False # 
    USE_GEMMA2 = False # 
    USE_QWEN = False # 
    USE_DEEPSEEK = True # 
    USE_DEEPSCALE = False # 

    TASK_GEN = True # for generative Text output task (suitable for RAG project)
    TEST_LLM = True
    USE_HUGGINGFACE = True # Pull model from Huggingface model hub
    USE_LMSTUIDO = False # for local LLM framework 
    USE_OLLAMA = False # for OLLAMA local LLM framework 
    USE_VLLM = False # for VLLM  LLM framework

    # mulitlingual LLM model 
    model1 = "meta-llama/Llama-3.2-3B-Instruct"  # llama3.2  3B-Instruct

    model2 =  "google/gemma-2-2b-it" # gemma 2 9B (mulitlingual)
    model3 = "Qwen/Qwen2.5-3B-Instruct" # Qwen 3B (mulitlingual)
    model4 = 'Qwen/Qwen2.5-7B-Instruct' # Qwen 7B (mulitlingual)
    model5 = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B" # DeepSeek Distill 1.5B (mulitlingual)
    model6 = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B" # DeepSeek Distill 7B (mulitlingual)
    model7 = "agentica-org/DeepScaleR-1.5B-Preview"

    # for VLM model
    vlmModel1 = "Qwen/Qwen2.5-VL-3B-Instruct"
    vlmModel2 = "Qwen/Qwen2.5-VL-7B-Instruct"


    # Mult Embedding model
    embedModel1 = 'intfloat/multilingual-e5-small' # for embedding model support chinese
    embedModel2 = "intfloat/multilingual-e5-large-instruct"
    embedModel3 = "Alibaba-NLP/gte-Qwen2-1.5B-instruct" # for embedding model support chinese
    embedModel4 = "Alibaba-NLP/gte-multilingual-base" # for embedding model support chinese
    embedModel5 = "BAAI/bge-m3" # for multilingual embedding model
    embedModel6 = "jinaai/jina-embeddings-v3"
    embedModel7 = "ollama/mxbai-embed-large"  # ollama 
    embedModel8 = "ollama/nomic-embed-text"  # ollama 
    

    # LLM for AI Agent  OLLAMA (for local deployment) ,requires pulling the ollama image and running the server locally
    llmModel1 = "ollama/deepseek-r1:8b"
    llmModel2 = "ollama/deepseek-r1:7b"
    llmModel3 = "ollama/qwen2.5"
    llmModel4 = "ollama/llama3.2"
    llm_base_url1 = "http://localhost:11434"

   
    # 



    FEW_SHOT_TEST= False#True
    USE_WANDB = True#True # for  LLM evalution and debug , track fine tuning performance

    USE_DEEPEVAL = True#False # for LLM evalution   
    USE_TRAIN =  False #True #False#True Much be use GPU for Training 
    
    # For VectorDB selection
    USE_FAISS = False#True # For RAG VectorDB
    USE_CHROMA = True #False #True #False # for RAG VectorDF
    USE_PINECONE = False#True#False #True # for RAG VectorDF
    USE_WEAVIATE = False#True #False # for RAG VectorDF
    USE_MILVUS = False#True              # for RAG VectorDF

    # for LLM fine tuning
    maxTrainData = 200#3500#5000 #10000#5000 #10000
    maxEvalData = 20#100 # 20 


    # LLM parameters
    reportTo ="none"
    topK = 40
    topP = 1.0
    temperature = 0.6 #0.5
    repetition_penalty = 1.05 # 1.1
    maxOutToken = 1024#180 #100
    

    
    maxToken=  512#768#512#768 # 512 for test only

In [49]:
from crewai import Agent, Task, Crew, Process
from crewai_tools import ScrapeWebsiteTool, SerperDevTool
from crewai.tools import BaseTool, tool
from crewai import LLM
import crewai
from pydantic import BaseModel, Field
from typing import List, Dict, Type
from crewai_tools import PDFSearchTool

In [50]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [51]:
from langchain_ollama import OllamaLLM, ChatOllama
from langchain_openai import ChatOpenAI


In [52]:
import ctypes
def clearMemory():
    for _ in range(5):
        torch.cuda.empty_cache()
        ctypes.CDLL("libc.so.6").malloc_trim(0)
        gc.collect()
        time.sleep(0.3)

In [53]:
# clearMemory()

In [54]:
## Define Test Document path 
pdfFilePath1 = "../test-document/Attention .pdf"
pdfFilePath2 = "../test-document/yolo.pdf"
pdfDir = "../test-document"

bankStatementDir = "../bank-statement-document/"
bankStatementSamples =  "../bank-statement-document/Bank-Statement-Template-2-TemplateLab.pdf"

## CrewAI Support LLM List :  LiteLLM 
- <https://docs.crewai.com/how-to/llm-connections>
- <https://docs.crewai.com/concepts/llms>



In [55]:
from crewai_tools import PDFSearchTool, YoutubeChannelSearchTool
from crewai_tools import RagTool

from crewai.tools import BaseTool, tool

# Chanin
from langchain_community.document_loaders import (TextLoader,
                                                  PyMuPDFLoader,
                                                  PyPDFDirectoryLoader,
                                                  PyPDFLoader)

In [ ]:
# # for ML Model
# if CFG.USE_WANDB:
#     # train report to  W&B tool
#     # import wandb
#     # reportTo= "wandb"
#     # my_secret = os.getenv("wandb_api_key") 
#     # wandb.login(key=my_secret) # login 
#     # wandb.init(
# else: 
#     reportTo = "none"# None

if CFG.USE_WANDB:
    import wandb
    # reportTo= "wandb"
    my_secret = os.getenv("wandb_api_key") 
    wandb.login(key=my_secret) # login 
    import weave
    # Initialize Weave with your project name 
    weave.init("crewai-ai-bank-statement-document-analysis-project")



wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/johnsonhk88/.netrc


# Define the tools for CrewAI agent

In [57]:
#Tool for AI agent

# pdfTool = PDFSearchTool(pdfDir=bankStatementDir)

In [58]:
# Custom PDF Search Tool for CrewAI
@tool("Custom PDF Extractor")
def pdf_extractor(
    pdf_dir: str
) -> list:
    """
    Extracts text from PDF files in the specified directory.
    Args:
        pdf_dir (str): The directory containing PDF files.
    Returns:
        List[Dict[str, str]]: A list of dictionaries containing the extracted text from each PDF file.
    """
    # results = pdfTool.search(query=query, pdf_dir=pdf_dir)
    loader = PyMuPDFLoader(pdf_dir)
    documents = loader.load()
    return documents

In [59]:
@tool("PDF directory Extractor")
def pdf_directory_extractor(
    pdf_dir: str = pdfDir
) -> list:
    """
    Extracts text from all PDF files in the specified directory.
    Parameters:
    - pdf_dir (str): The directory containing PDF files.
    Returns:
    - List of dictionaries containing the content and source of each PDF file.
    """
    loader = PyPDFDirectoryLoader(pdf_dir)
    documents = loader.load()
    
    # results = []
    # for doc in documents:
    #     if query.lower() in doc.page_content.lower():
    #         results.append({"content": doc.page_content, "source": doc.metadata.get("source", "")})
    
    # return results

    return documents

In [60]:
@tool("RAG Tool")
def rag_tool(query : str) -> str:
    """Tool to search 
    """
    pass
    # Encode the question using the embedding model
    # query_vec = model.encode(question)
    
    # # Get top 5 similar vector    # Encode the question
    # query_vec = model.encode(question)
    
    # # Get top 5 simil`ar vector
    # results = company_db.search(query_vec, top_k = 5)

    # # Build context from the results
    # context = "\n".join([f"- {res['metadata']['sentence']}" for res in results])

    # # Create the prompt
    # prompt = f"""You are a helpful assistant. Use the context below to answer the user's question.

    #         Context:
    #         {context}

    #         Question: {question}

    #         Answer:
    #         """

    # # Generate an answer using the context
    # client = OpenAI()

    # response = client.responses.create(
    #     model = "gpt-4o-mini",
    #     input = prompt
    # )

    # answer = response.output_text
    
    # # Return the answer
    # return answer


In [61]:
class DocumentExtractInput(BaseModel):
    query: str = Field(... , description="")


class DocumentSearchTool(BaseTool):
    name: str = "DocumentSearchTool"
    description: str = "Search the document for the given query."
    args_schema: Type[BaseModel] = DocumentExtractInput


    
    





In [81]:
class PDFSearchToolInput(BaseModel):
    query: str = Field(description="The search query.")

        
class PDFExtractTool(BaseTool):
    name : str =  "PDFSearchTool"
    description: str  = "A tool to search for information in PDF files."
    args_schema: Type[BaseModel] = PDFSearchToolInput
    
    # model_config = ConfigDict(
    #     arbitrary_types_allowed=True,
    # )

    def __init__(self, pdf_dir: str):
        """Initialize the PDF Search Tool."""
        super().__init__()
        self.pdf_dir = pdf_dir
        # initalize VectorDB 
        self.vectorDB = Chroma.from_documents(documents, embedding_function=embedModel1)
        # document process
        self._process_documents(self.pdf_dir)




    def _process_documents(self, pdf_dir: str):
        """Process the documents in the specified directory."""
        # loader = PyPDFDirectoryLoader(pdf_dir)
        # documents = loader.load()
        loader = PyMuPDFLoader(pdf_dir)
        documents = loader.load() # list of document 
        # 
        
        

        

    def _run(self, query: str) -> str:
        return "test"

In [63]:
llm = LLM(
    # model= CFG.llmModel2,
    model= CFG.llmModel3,
    base_url= CFG.llm_base_url1,
)

## Define AI Agent

In [64]:
# 
document_agent = Agent( role="Document Processing Specialist",
    goal="Extract information from a document",
    backstory="You are a document extraction agent. You will be given a document and you need to extract the relevant information from it.",
    # tools=[webSearchTool],
    tools=[pdf_extractor], # Custom PDF Search Tool]
    llm=llm,
    max_iter=10,
    max_execution_time=120, # seconds
    allow_delegation= False,
    verbose=True,

)


In [65]:
data_structuring_agent = Agent(
    role='Data Structuring Expert',
    goal='Parse and structure extracted text into a usable transaction format.',
    backstory="Expert in financial data organization.",
    # tools=[text_tool]
    llm=llm,
    max_iter=10,
    max_execution_time=120, # seconds
    allow_delegation= False,
    verbose=True,
)


In [66]:
financial_analyst_agent = Agent(
    role='Financial Analyst',
    goal='Analyze transaction data and identify key financial insights.',
    backstory="Experienced financial analyst.",
    verbose=True,
    # tools=[analysis_tool]
)

In [67]:
# rag_agent = Agent( role="RAG Agent",
#                   goal="Extract information from a document",
#                  backstory=("You are a document retrieval agent."
#                               "You will be given a document and you need to extract the relevant information from it."),
#                  llm=llm,
#                  allow_delegation= True,
#                  verbose=True,
# )

In [68]:
# fiancial_agent = Agent( role= "Personal Finance Analystic Expert",
#                         goal="Extract information from a document",
#                         backstory=("You are a document retrieval agent."
#                                    "You will be given a document and you need to extract the relevant information from it."),
#                         # tools=[webSearchTool],
#                         llm=llm,
#                         max_iter=10,
#                         allow_delegation= True,
#                         verbose=True,
# )

In [69]:
# summary_agent = Agent( role="Summarization Agent",
#                       goal="Summarize the document",
#                         backstory="You are a document summarization agent. You will be given a document and you need to summarize it.",

# )

In [70]:
report_agent = Agent( role= "Report Writter Expert for Financial Analystic",
                      goal="Generate Clear and Concise Report from the analysis results",
                      backstory=("Professional report writer with expertise in financial analysis."),
                     
                     )





# Task for AI Agent

In [71]:
load_document_task = Task(
    description="load the document from directory:  {pdf_dir} ",
    expected_output=("document content in JSON format with key: content, source , page_number"
                      "output the document content in list of PDF page content"
                      "Output JSON format of the extracted text from the PDF files in the directory with key: content, source , page_number"),
    agent=document_agent,  
)

In [72]:
retrieve_content_task = Task(
    description="Retrieve the content from the document",
    expected_output="Output the content of the document",
    agent=data_structuring_agent
)

In [73]:
Financial_Analytic_task = Task(
    description="Extract the financial information from the document, ",
    expected_output="Output the financial information from the document",
    agent=financial_analyst_agent
)

In [74]:
summarize_task = Task(

    description="Summarize the document",
    expected_output="Output the summary of the document",
)

In [75]:
report_task = Task(

    description="Generate a report from the document",
    expected_output="Output the report of the document",
)

In [76]:
crew1 = Crew(
    tasks=[
        load_document_task,
        retrieve_content_task,
        # Financial_Analytic_task,
        # summarize_task,
        # report_task
    ],
    agents=[
        document_agent,
        data_structuring_agent,
        # financial_analyst_agent,
        # rag_agent,
        # fiancial_agent,
        # summary_agent,
        # report_agent
    ],
    # max_concurrent_tasks=2, # limit the number of concurrent tasks
    verbose=True, 
)


In [77]:
inputs = {
    "pdf_dir" : bankStatementSamples

}

In [78]:
result = await crew1.kickoff_async(inputs= inputs) 

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 84b34ee6-4140-4730-b871-901f543d6f0c                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 9e946abe-ebb6-409b-8490-ab96fd8cb9ac
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 9e946abe-ebb6-409b-8490-ab96fd8cb9ac
       Status: Executing Task...
    └── 🤖 Agent: Document Processing Specialist
            Status: In Progress

# Agent: Document Processing Specialist
## Task: load the document from directory:  ../bank-statement-document/Bank-Statement-Template-2-TemplateLab.pdf 


🚀 Crew: crew
└── 📋 Task: 9e946abe-ebb6-409b-8490-ab96fd8cb9ac
       Status: Executing Task...
    └── 🤖 Agent: Document Processing Specialist
            Status: In Progress
        └── 🧠 Thinking...

...

🚀 Crew: crew
├── 📋 Task: 9e946abe-ebb6-409b-8490-ab96fd8cb9ac
│      Assigned to: Document Processing Specialist
│      Status: ✅ Completed
│   └── 🤖 Agent: Document Processing Specialist
│           Status: ✅ Completed
│       └── 🔧 Used Custom PDF Extractor (1)
└── 📋 Task: 52be383d-8d7a-447a-8d52-f61565d218fc
       Assigned to: Data Structuring Expert
       Status: ✅ Completed
    └── 🤖 Agent: Data Structuring Expert
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 52be383d-8d7a-447a-8d52-f61565d218fc                                                                     │
│  Agent: Data Structuring Expert                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

/home/johnsonhk88/.local/lib/python3.10/site-packages/weave/trace/object_record.py:123: PydanticDeprecatedSince20: The `__fields__` attribute is deprecated, use `model_fields` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  value = getattr(object, key)
/home/johnsonhk88/.local/lib/python3.10/site-packages/weave/trace/object_record.py:123: PydanticDeprecatedSince20: The `__fields_set__` attribute is deprecated, use `model_fields_set` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  value = getattr(object, key)


╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 84b34ee6-4140-4730-b871-901f543d6f0c                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

/home/johnsonhk88/.local/lib/python3.10/site-packages/weave/trace/object_record.py:123: PydanticDeprecatedSince20: The `__fields__` attribute is deprecated, use `model_fields` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  value = getattr(object, key)
/home/johnsonhk88/.local/lib/python3.10/site-packages/weave/trace/object_record.py:123: PydanticDeprecatedSince20: The `__fields_set__` attribute is deprecated, use `model_fields_set` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  value = getattr(object, key)


In [79]:
print(result.model_dump_json())

{"raw":"Issue Date:\nPeriod:\nAccount Activity\n\nDate Payment Type Paid In Paid Out Balance\n\nYour Account Statement Detail Note:\n\nPrint Form Save Form Reset Form \n\nBranch Name 231 Valley Farms Street Santa Monica, CA bickslowbank@domain.com mm/dd/yyyy mm/dd/yyyy to mm/dd/yyyy 111-234-567-890 Bit Manufacturing Ltd 2450 Courage St, STE 108 Brownsville, TX 78521 Balance Brought Forward 8,313.30 mm/dd/yyyy Fast Payment Amazon 132.30 8,181.00 mm/dd/yyyy BACS eBAY Trading Co. 515.22 7,665.78 mm/dd/yyyy Fast Payment Morrisons Petrol 80.00 7,585.78 mm/dd/yyyy BACS Business Loan 20,000.00 27,585.78 mm/dd/yyyy BACS Jumes White Media 2,416.85 25,168.93 mm/dd/yyyy Fast Payment ATM High Street 100.00 25,068.93 mm/dd/yyyy BACS Accorn Advertising Studios 150.00 24,918.93 Fast Payment mm/dd/yyyy Marriott Hotels 177.00 24,741.93 mm/dd/yyyy Fast Payment Abelio Scotrail Ltd 122.22 24,619.71 mm/dd/yyyy Fast Payment Cheque 000234 1,200.00 23,419.71 mm/dd/yyyy Int. Bank Interest Paid 9.33 23,429.04 m

In [80]:
print(result.raw)

Issue Date:
Period:
Account Activity

Date Payment Type Paid In Paid Out Balance

Your Account Statement Detail Note:

Print Form Save Form Reset Form 

Branch Name 231 Valley Farms Street Santa Monica, CA bickslowbank@domain.com mm/dd/yyyy mm/dd/yyyy to mm/dd/yyyy 111-234-567-890 Bit Manufacturing Ltd 2450 Courage St, STE 108 Brownsville, TX 78521 Balance Brought Forward 8,313.30 mm/dd/yyyy Fast Payment Amazon 132.30 8,181.00 mm/dd/yyyy BACS eBAY Trading Co. 515.22 7,665.78 mm/dd/yyyy Fast Payment Morrisons Petrol 80.00 7,585.78 mm/dd/yyyy BACS Business Loan 20,000.00 27,585.78 mm/dd/yyyy BACS Jumes White Media 2,416.85 25,168.93 mm/dd/yyyy Fast Payment ATM High Street 100.00 25,068.93 mm/dd/yyyy BACS Accorn Advertising Studios 150.00 24,918.93 Fast Payment mm/dd/yyyy Marriott Hotels 177.00 24,741.93 mm/dd/yyyy Fast Payment Abelio Scotrail Ltd 122.22 24,619.71 mm/dd/yyyy Fast Payment Cheque 000234 1,200.00 23,419.71 mm/dd/yyyy Int. Bank Interest Paid 9.33 23,429.04 mm/dd/yyyy DD OVO E